<a href="https://colab.research.google.com/github/Minuteman-Library-Network/Loan-Rule-Database/blob/main/Loan_Rule_Modeler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Jeremy Goldstein Minuteman Library Network

This Python script is intended to help test the loan rule determiner in Sierra to ensure that checkouts are being assigned to the expected loan rule.

The script uses a database containing an extract of the loan rule and loan rule determiner tables, which is updated regularly via a manual process so may not reflect recent changes.

# Instructions

Running the script

Each code block can be run one at a time by clicking the play icon that appears when you hover over the [] marker. Once that block finishes running a green check will appear to the left of the block and any output from that portion of the script will appear beneath it, along with any errors that may be encountered.

You may also use the "Run All" function, found within the Runtime drop down menu to Run the entire script, though there are points where user input is required before the script will continue past a particular code block to be on the lookout for, these are indicated by the words "Action Required".

You may reset the output by going to the Edit menu and selecting clear all outputs.

# Step 1: Configure Python/Colab Environment

import Python libraries that will be used within the script

**Action Required** The first time during a session in which you run the script you will have to grant permission for the script to access your Google Drive.  This is so that it may retrieve the loan rule database stored in the Minuteman Document Center.

In [ ]:
#@title
import sqlite3
import pandas as pd
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

# Step 2: Select Your Parameters

**Action Required** Select a combination of [item type](https://docs.google.com/document/d/1s29TgiBwwQvs-_jX4a4exgiGdpMucgX-2SWRlCB4C24/edit?usp=sharing), [patron type](https://docs.google.com/document/d/11I7VX4rAC3KC5-26vJH8kszwvGMxK3b_EgMKtaU1Pz0/edit?usp=sharing) and checkout location from the drop down menus below.  Ptype 9999 represents any ptype.

Once the code block has been run, you may change your menu selections without having to click on run again.  However you will need to rerun the code block for Step 3.

In [ ]:
#@title Select an iType, pType and Checkout Location { run: "auto", vertical-output: true, display-mode: "form" }
itype = '0' #@param ["0","1","2","3","4","5","6","7","8","9","10","11","12","13","19","20","21","22","23","24","25","26","27","28","29","30","31","32","33","34","35","36","37","38","39","40","41","42","43","44","45","46","47","48","50","51","52","100","101","102","103","104","105","106","107","108","109","113","114","115","116","117","118","119","120","121","122","123","124","125","126","127","128","129","130","131","132","133","150","151","152","153","154","155","156","157","158","159","160","163","164","165","166","167","168","169","170","171","172","173","174","175","176","177","178","179","180","181","182","183","221","222","223","224","239","241","242","243","244","245","246","247","248","249","250","251","252","253","255"]
ptype = '9999' #@param ["9999","1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16","17","18","20","21","22","23","24","26","27","29","30","31","32","33","34","35","36","37","38","39","40","41","42","43","44","45","46","47","106","110","115","116","117","120","121","122","126","129","130","133","137","141","147","159","163","166","175","194","195","197","199","200","201","202","203","204","205","206","207","254","255"]
location = 'act' #@param ["act","arl","ash","bed","blm","brk","cam","con","ddm","dea","dov","fpl","frk","fst","hol","las","lex","lin","may","med","mil","mld","mls","mwy","nat","nee","nor","ntn","oln","pmc","reg","shr","som","sto","sud","wat","wel","win","wlm","wob","wsn","wwd","wyl"]

# Step 3: Result

Run this last code block to query the loan rule database and see the loan rule used by your selected combination of itype, ptype and checkout location.

In [ ]:
#@title
loan_rules = '/gdrive/Shareddrives/SIC Document Center/Reports and Reporting Tools/loan_rules.db'

try:
    sqliteConnection = sqlite3.connect(loan_rules)
    cursor = sqliteConnection.cursor()
    
    sql_query = """
                SELECT *
                FROM(
                SELECT
                l.LoanRuleNum,
				        l.name,
                d.id,
                d.location,
                d.itype,
                d.ptype,
				        CASE
			        		WHEN l.code = 'R' THEN 'R-Days'
			        		WHEN l.code = 'K' THEN 'K-Hours no overnight'
			        		WHEN l.code = 'J' THEN 'J-Hours overnight'
			        		WHEN l.code = 'N' THEN 'N-Non-circulating'
			        		WHEN l.code = 'I' THEN 'I-Hours rounded to minute'
			        	END AS 'code',
                l.NormalLoanPeriod,
                l.Holdable,
				        l.MaxNumberOfRenewals,
			        	l.GracePeriodForFines,
                CASE
				        	WHEN l.GracePeriodForFines = '0' THEN l.AmountOfFine1stFPer
				        	ELSE l.AmountOfFine2NDFPer
                END AS AmountOfFine,
				        CASE
				        	WHEN l.GracePeriodForFines = '0' THEN l.NumberOfDHR1stFPer
					        ELSE l.NumberOfDHR2ndFPer
                END AS LengthOfFinePeriod,
				        l.DefaultItemCost

                FROM
                determiner_expanded d
                JOIN
                loan_rule l
                ON
                d.rule_number = l.LoanRuleNum
                WHERE 
                d.itype IN('"""+ itype +"""','9999') 
                AND (d.location LIKE substr('"""+ location +"""',1,2)||'%' OR d.location = '?????')
                AND d.ptype IN ('"""+ ptype +"""','9999')
                )a
                ORDER BY a.id DESC
                LIMIT 1
                """
    cursor.execute(sql_query)
    sqliteConnection.commit()
    loan_rows = cursor.fetchall()
    cursor.close()
except sqlite3.Error as error:
    print("Failed to run query", error)

column_names = ["LoanRuleNum","Name","DeterminerNum","Location","IType","PType","Code","NormalLoanPeriod","Holdable","MaxNumberOfRenewals","GracePeriodForFines","AmtOfFine","LengthOfFinePeriod","DefaultItemCost"]
loan_df = pd.DataFrame(loan_rows, columns=column_names)
print('Here is the loan rule used for this checkout:')
loan_df